## Pandas analysis

In the following a series of exercises is proposed on a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGA's. Each measurement (i.e. each raw) consists of the address of the TDC providing the signal, 'FPGA' and 'TDC_Channel, and the timing information itself, 'ORBIT_CNT', 'BX_COUNTER' and 'TDC_MEAS'. Each TDC count correspond 25/30 ns, whereas the BX_COUNTER feauters gets updated every 25 ns and the ORBIT_CNT every 'x' BX_COUNTER. You can see these way of storing the time as similar to hours, minutes and seconds.

1\. Create a Pandas DataFrame by read N raws of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of raws and larger that 10k.

2\. Find out the value of 'x'

3\. Find out how much the data taking lasted. You can either make an estimate on the baseis of the fraction of the measurements (raws) you read, or perform this check precisely by reading out the whole dataset

4\. Create a new column with the actual time in ns (as a combination of the other three columns with timing information)

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1

6\. Create a new DataFrame with only the raws with HEAD=1

7\. Make two occupancy plots (one per FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [39]:
import pandas as pd
import numpy as np

# 1 - Load data from file
f_name = "~/data/data_000637.txt"
data = pd.read_csv(f_name, nrows = 10**4)

# 2 - Find value of x
counter = data.loc[:,['BX_COUNTER']]
x = counter.max() + 1
print(x)

# 3 - Find out duration time
def compute_time(index):
    result = data['ORBIT_CNT'].iloc[index]*x + data['TDC_MEAS'].iloc[index]*25/30 + data['BX_COUNTER'].iloc[index]*25
    return result

time_first_sample = compute_time(0)
time_last_sample = compute_time(-1)
total_time = time_last_sample - time_first_sample
print("Total time:", total_time)

# 4 - Add new column 


BX_COUNTER    3564
dtype: int64
Total time: BX_COUNTER    316464.167969
dtype: float64
